In [10]:
import pyodbc
import pandas as pd
import re

In [ ]:
def fetch_and_clean_descriptions(*ar_numbers):

    # Normalize input to list

    ar_numbers=list(ar_numbers)
    
 
    # Connection string for Oracle database

    conn_str = (

        f"DRIVER={{Oracle in OraClient19Home1}};"

        f"DBQ=AS9NUCRP;"

        f"UID=;"

        f"PWD=;"

    )
 
    # Format AR numbers for SQL IN clause

    formatted_ar_numbers = ', '.join([f"'{ar}'" for ar in ar_numbers])
 
    query = f"""

    SELECT

        AR_NUMBER,

        RTRIM(

            REPLACE(

                REPLACE(

                    XMLAGG(XMLELEMENT("x", DESCRIPTION_NOTES || ' ') ORDER BY GEN_ARG).GetClobVal(),

                '<x>', ''),

            '</x>', ''), ' ') AS FULL_DESCRIPTION

    FROM TIDARCOM

    WHERE AR_NUMBER IN ({formatted_ar_numbers})

      AND AR_COMMENT_TYPE = 'D'

      AND TRIM(DESCRIPTION_NOTES) IS NOT NULL

    GROUP BY AR_NUMBER

    """
 
    # Executing query

    conn = pyodbc.connect(conn_str)

    cursor = conn.cursor()

    cursor.execute(query)

    rows = cursor.fetchall()

    conn.close()
 
    # Converting to DataFrame

    data = [{"AR_NUMBER": row[0], "FULL_DESCRIPTION": row[1]} for row in rows]

    df = pd.DataFrame(data)
 
    # Clean function

    def clean_description(desc):

        desc = re.sub(r'([+*]+)', ' ', desc)

        desc = re.sub(r'[^\x00-\x7F]+', '', desc)

        desc = re.sub(r'\s+', ' ', desc)

        return desc.strip()
 
    df['CLEANED_DESCRIPTION'] = df['FULL_DESCRIPTION'].apply(clean_description)
 
    # Save to CSV

    output_file = 'output_descriptions.csv'

    df[['AR_NUMBER', 'CLEANED_DESCRIPTION']].to_csv(output_file, index=False)
 
    print(f"\n Cleaned output saved to: {output_file}\n")

    print(df[['AR_NUMBER', 'CLEANED_DESCRIPTION']])

    return df


In [ ]:
fetch_and_clean_descriptions('00000051', '04788102','00002283')  